# Generate simulated data to link

In this case study, we imagine running PVS on the 2030 Census Unedited File (CUF) -- see the main notebook for more details, including references used throughout this notebook.
This notebook creates input (CUF) and reference files approximating what would be used in such a PVS process.

In [1]:
import pseudopeople as psp
import os
import logging
import warnings
import os
import shutil
from pathlib import Path
# Importing pandas for access, regardless of whether we are using it as the compute engine
import pandas
import numpy

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# DO NOT EDIT if this notebook is not called generate_simulated_data_small_sample.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
# If running with the default parameters, you can overwrite this notebook; otherwise,
# save it to another filename.
# TODO: Rename the notebook to omit 'small_sample' in the filename and omit all outputs
# from the 'canonical version'
data_to_use = 'small_sample'
output_dir = 'output'
compute_engine = 'pandas'
num_jobs = 10
cpus_per_job = 2
memory_per_job = "10GB"

In [4]:
! date

Mon Nov 20 10:15:02 AM PST 2023


In [5]:
psp.__version__

'0.8.3.dev6+g31db93a'

In [6]:
def start_dask_distributed_over_slurm():
    from dask_jobqueue import SLURMCluster

    cluster = SLURMCluster(
        queue='long.q',
        account="proj_simscience",
        # If you give dask workers more than one core, they will use it to
        # run more tasks at once, which can use more memory than is available.
        # To have more than one thread per worker but use them all for
        # multi-threading code in one task
        # at a time, you have to set cores=1, processes=1 and job_cpu > 1.
        cores=1,
        processes=1,
        memory=memory_per_job,
        walltime="10-00:00:00",
        # Dask distributed looks at OS-reported memory to decide whether a worker is running out.
        # If the memory allocator is not returning the memory to the OS promptly (even when holding onto it
        # is smart), it will lead Dask to make bad decisions.
        # By default, pyarrow uses jemalloc, but I could not get that to release memory quickly.
        # Even this doesn't seem to be completely working, but in combination with small-ish partitions
        # it seems to do okay -- unmanaged memory does seem to shrink from time to time, which it wasn't
        # previously doing.
        job_script_prologue="export ARROW_DEFAULT_MEMORY_POOL=system\nexport MALLOC_TRIM_THRESHOLD_=0",
        job_cpu=cpus_per_job,
        # NOTE: This is, as Dask requests, a directory local to the compute node.
        # But IHME's cluster doesn't support this very well -- it can be small-ish,
        # full of stuff from other users, etc.
        local_directory=f"/tmp/{os.environ['USER']}_dask_generate_simulated_data",
        # NOTE: Network file system -- probably slow and doing a lot of unnecessary I/O!
        # local_directory=f"/ihme/scratch/users/{os.environ['USER']}/dask_work_dir/dask_generate_simulated_data",
        # HACK: Avoid nodes with /tmp too full (as of 11/17/2023)
        job_extra_directives=["-x long-slurm-sarchive-p00[53-59]"],
        log_directory=f"/ihme/temp/slurmoutput/{os.environ['USER']}",
    )

    cluster.scale(n=num_jobs)
    # Supposedly, this will start new jobs if the existing
    # ones fail for some reason.
    # https://stackoverflow.com/a/61295019
    cluster.adapt(minimum_jobs=num_jobs, maximum_jobs=num_jobs)

    from distributed import Client
    client = Client(cluster)

    client.wait_for_workers(n_workers=num_jobs)

    return cluster, client

if compute_engine == 'pandas':
    import pandas as pd
elif compute_engine == 'dask':
    import dask
    # HACK: Use Python instead of pyarrow strings; this will usually be much slower and
    # require more memory, but pyarrow string columns have a 2GB max
    # Worked around this using large_strings in pyarrow instead
    # dask.config.set({"dataframe.convert-string": False})
    # Make Dask much less conservative with memory management: don't start spilling
    # until over 85%, don't kill until basically at memory limit
    # (I don't much mind whether Dask or slurm kills a worker)
    # We want to avoid spilling if at all possible, since it uses a resource
    # (local disk space) which is unpredictably allocated and running out of
    # it can cause the whole computation to fail
    dask.config.set({"distributed.worker.memory.target": False})
    dask.config.set({"distributed.worker.memory.spill": 0.85})
    dask.config.set({"distributed.worker.memory.pause": 0.85})
    dask.config.set({"distributed.worker.memory.terminate": 0.975})

    cluster, client = start_dask_distributed_over_slurm()

    import dask.dataframe as pd

    display(client)
elif compute_engine.startswith('modin'):
    if compute_engine.startswith('modin_dask_'):
        import modin.config as modin_cfg
        modin_cfg.Engine.put("dask") # Use dask instead of ray (which is the default)

        if compute_engine == 'modin_dask_distributed':
            cluster, client = start_dask_distributed_over_slurm()
        else:
            from distributed import Client
            cpus_available = int(os.environ['SLURM_CPUS_ON_NODE'])
            client = Client(n_workers=int(cpus_available / 2), threads_per_worker=2)

        # Why is this necessary?!
        # For some reason, if I don't set NPartitions, it seems to default to 0?!
        num_row_groups = 1 if data_to_use == 'small_sample' else 334
        modin_cfg.NPartitions.put(min(num_jobs * 5, num_row_groups))
        modin_cfg.MinPartitionSize.put(1_000) # ensure no column-axis partitions -- they'll need to be joined up right away anyway by our row-wise noising

        display(client)
    elif compute_engine == 'modin_ray':
        # Haven't worked on distributing this across multiple nodes
        import ray
        ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}}, num_cpus=int(os.environ['SLURM_CPUS_ON_NODE']))
    else:
        # Use serial Python backend (good for debugging errors)
        import modin.config as modin_cfg
        modin_cfg.IsDebug.put(True)

    import modin.pandas as pd

    # https://modin.readthedocs.io/en/stable/usage_guide/advanced_usage/progress_bar.html
    from modin.config import ProgressBar
    ProgressBar.enable()

In [7]:
# Helpers for dealing with lazy evaluation -- Dask doesn't actually compute
# anything until you explicitly tell it to, while Pandas and Modin are eager

def persist(*args):
    if compute_engine == 'dask':
        if len(args) == 1:
            return client.persist(args[0])

        return client.persist(args)
    else:
        # Eagerly computed already
        if len(args) == 1:
            return args[0]

        return args

def compute(obj):
    if compute_engine == 'dask':
        return obj.compute()
    else:
        # Eagerly computed already
        return obj

In [8]:
# DataFrame operations that need to be done in specific ways for Dask

def drop_duplicates(df, subset=None, sort_col=None, keep='last'):
    original_columns = list(df.columns)

    if subset is None:
        subset = original_columns.copy()
    elif isinstance(subset, str):
        subset = [subset]
    else:
        subset = list(subset)

    if sort_col is not None:
        df = df.sort_values(sort_col)

    if compute_engine == 'pandas' or compute_engine.startswith('modin'):
        return df.drop_duplicates(subset=subset, keep=keep)
    elif compute_engine == 'dask':
        # NOTE: This approach depends crucially on https://github.com/dask/dask/issues/8437, as described in
        # https://github.com/dask/dask/issues/8437#issuecomment-983440465
        index_before = None
        if df.index.name is not None:
            index_before = df.index.name
            df = df.reset_index()

        if len(subset) == 1:
            # Cannot set_index with a column that contains any null values. Any rows that have nulls in any of subset
            # are by definition not duplicates.
            temp_index = subset[0]
            df = concat_avoid_fragmentation([
                df[df[temp_index].isnull()],
                df[df[temp_index].notnull()].set_index(temp_index).map_partitions(lambda x: x[~x.index.duplicated(keep=keep)]).reset_index(),
            ], ignore_index=True)
        else:
            # NOTE: This means it is best to put a high-cardinality column as the first item of subset
            temp_index = subset[0]
            # Cannot set_index with a column that contains any null values. Any rows that have nulls in any of subset
            # are by definition not duplicates.
            df = concat_avoid_fragmentation([
                df[df[temp_index].isnull()],
                df[df[temp_index].notnull()].set_index(temp_index).map_partitions(lambda x: x.reset_index().drop_duplicates(subset=subset, keep=keep).set_index(temp_index)).reset_index(),
            ], ignore_index=True)

        if index_before is None:
            return df
        else:
            return df.set_index(index_before, sort=False)
        # NOTE: The following is another approach I tried. It turns out that Dask groupbys don't work the way
        # you might expect for small groups, which is described more in groupby_agg_small_groups.
        # But even after working around that, it turned out to be much simpler to use the index-based approach above.
#     elif compute_engine == 'dask':
#         if sort_col is None:
#             df = df.assign(dummy_for_cumsum=1).assign(drop_duplicates_unique_id=lambda df: df.dummy_for_cumsum.cumsum()).drop(columns=['dummy_for_cumsum'])
#             sort_col = 'drop_duplicates_unique_id'

#         if keep == 'last':
#             to_keep = df.groupby(subset, dropna=False)[sort_col].max()
#         elif keep == 'first':
#             to_keep = df.groupby(subset, dropna=False)[sort_col].min()
#         else:
#             raise ValueError()

#         result = df.merge(to_keep.to_frame(), on=(subset + [sort_col]), how='inner')[original_columns]

#         if sort_col == 'drop_duplicates_unique_id':
#             return result
#         else:
#             # No guarantee of uniqueness
#             return drop_duplicates(result, subset=(subset + [sort_col]), keep='last')
    else:
        raise ValueError()

# NOTE: Dask groupbys don't work the way you might expect for small groups.
# In our application, when we groupby, we are usually grouping by a column (set)
# with very high cardinality -- almost as many groups as we have rows.
# Dask's agg function tries to create a data structure that is O(N) with the number of
# groups on a single node, which OOMs in this situation.
# Instead, we take advantage of https://github.com/dask/dask/issues/8437 again to turn
# this into a P2P shuffle operation that never holds any substantial amount of the data
# in any one place.
def groupby_agg_small_groups(df, by, agg_func):
    if compute_engine == 'pandas' or compute_engine.startswith('modin'):
        return agg_func(df.groupby(by))
    elif compute_engine == 'dask':
        if isinstance(by, str):
            by = [by]
        else:
            by = list(by)

        index_before = None
        if df.index.name is not None:
            index_before = df.index.name
            df = df.reset_index()

        # NOTE: This means it is best to put a high-cardinality column as the first item of by
        temp_index = by[0]

        # Cannot set_index with a column that contains any null values. Any rows that have nulls in any of subset
        # are not put into any group, like the default pandas behavior
        return df[df[temp_index].notnull()].set_index(temp_index).map_partitions(lambda x: agg_func(x.reset_index().groupby(by)))
    else:
        raise ValueError()

def concat_avoid_fragmentation(*args, **kwargs):
    result = pd.concat(*args, **kwargs)

    if compute_engine == 'dask' and result.npartitions > (num_jobs * 20):
        # By default, a Dask concat operation of A and B will lead to
        # a result with A.npartitions + B.npartitions partitions.
        # We do several operations that look like
        # df = concat([transformation_1(df), transformation_2(df)])
        # which doubles the number of partitions.
        # If we don't repartition, this doubling leads to a partition explosion,
        # which scales scheduler overhead and the memory size of the task graph.
        result = result.repartition(npartitions=(num_jobs * 5))

    return result

In [9]:
# NOTE: By default, Dask uses PyArrow string dtypes, not Python ones.
# This is great, because they are faster to work with, more memory efficient, and
# (crucially) nullable -- all of our string columns can be missing, which we represent
# in NumPy land with NaN, but that is finicky with Parquet.
# Unfortunately, the default PyArrow string dtype has a limit of 2GB of data per
# PyArrow "chunk," and although PyArrow chunks are supposed to work invisibly to the
# user, there are a number of bugs in PyArrow that cause common operations to try to
# switch an entire array to be a single chunk.
# See https://github.com/dask/dask/issues/10139#issuecomment-1812817180 for more on this.
# Due to these issues, we were running into the 2GB limit.
# PyArrow has a "large_string" dtype that has effectively no limit on size (64 bit instead
# of 32 bit offset).
# Dask lets us use this dtype, so long as we use it for *all* strings (due to a bug in Dask,
# see https://github.com/dask/dask/issues/10139#issuecomment-1812969372).

# Based on https://github.com/dask/dask/blob/b2f11d026d2c6f806036c050ff5dbd59d6ceb6ec/dask/dataframe/_pyarrow.py#L64-L98
# and code referenced from there
import pyarrow as pa

def is_pyarrow_string_dtype(dtype):
    """Is the input dtype a pyarrow string?"""

    pa_string_types = [pandas.StringDtype("pyarrow"), pandas.ArrowDtype(pa.string()), pandas.ArrowDtype(pa.large_string())]
    return dtype in pa_string_types

def is_pyarrow_string_index(x):
    if isinstance(x, pandas.MultiIndex):
        return any(is_pyarrow_string_index(level) for level in x.levels)
    return isinstance(x, pandas.Index) and is_pyarrow_string_dtype(x.dtype)

def to_pyarrow_large_string(df):
    string_dtype = pandas.ArrowDtype(pa.large_string())

    # Possibly convert DataFrame/Series/Index to string_dtype
    dtypes = None
    if isinstance(df, pandas.DataFrame):
        dtypes = {
            col: string_dtype for col, dtype in df.dtypes.items() if is_pyarrow_string_dtype(dtype)
        }
    elif dtype_check(df.dtype):
        dtypes = string_dtype

    if dtypes:
        df = df.astype(dtypes, copy=False)

    # Convert DataFrame/Series index too
    if is_pyarrow_string_index(df.index):
        if isinstance(df.index, pandas.MultiIndex):
            levels = {
                i: level.astype(string_dtype)
                for i, level in enumerate(df.index.levels)
                if is_pyarrow_string_dtype(level.dtype)
            }
            # set verify_integrity=False to preserve index codes
            df.index = df.index.set_levels(
                levels.values(), level=levels.keys(), verify_integrity=False
            )
        else:
            df.index = df.index.astype(string_dtype)
    return df

def increase_string_capacity(df):
    if compute_engine != 'dask':
        # Not using pyarrow strings by default
        return df

    return df.map_partitions(
        # NOTE: In Dask they use enforce_metadata=False
        to_pyarrow_large_string, token="to_pyarrow_large_string"
    )

In [10]:
def remove_path(path):
    path = Path(path)
    if path.is_file():
        os.remove(path)
    elif path.exists():
        shutil.rmtree(path)

def save_file_with_ground_truth(file_name, file, ground_truth):
    # Check that file and ground truth have the same records
    assert (
        # record_id is unique
        len(file) ==
        len(drop_duplicates(file[['record_id']]))
    )
    assert (
        len(ground_truth) ==
        len(file[['record_id']].merge(ground_truth[['record_id']], on='record_id', how='inner'))
    )

    file_path = f'{output_dir}/{data_to_use}/{file_name}.parquet'
    remove_path(file_path)
    file.to_parquet(file_path)

    ground_truth_path = f'{output_dir}/{data_to_use}/{file_name}_ground_truth.parquet'
    remove_path(ground_truth_path)
    ground_truth.to_parquet(ground_truth_path)

## Load pseudopeople simulated datasets

### Record ID tracking (data lineage)

We do a little bit of work here to enable tracking the "ground truth" (the simulant IDs from
pseudopeople).
We give each pseudopeople record/row a unique identifier for tracking, and then we immediately
separate the ground truth information (which we would not have if we were using real data)
from the rest of the columns (which we would have).
The ground truth is only used in the specific "ground truth" section of this notebook,
to help avoid accidentally leaking information into the case study.

Since we also combine/aggregate pseudopeople records as part of the process of generating the
simulated PVS reference files, ground truth is a bit more complicated than you might imagine.
For example, the ground truth may tell us that a single row in a reference file is actually
a composite of several individuals, because even the deterministic linkage (by SSN) we use
here is not without error.

We handle this by tracking *all* source records used in the construction of each record in
our reference files.
This is achieved by having a table mapping composite record IDs to the "source record IDs"
(IDs of records that were directly generated by pseudopeople).
When we aggregate records, this is combined accordingly.

In [11]:
def add_unique_id_col(df, col_name='unique_id', value_prefix=''):
    if compute_engine == 'pandas' or compute_engine.startswith('modin'):
        return df.reset_index().rename(columns={'index': col_name}).assign(**{col_name: lambda df: value_prefix + df[col_name].astype(str)})
    elif compute_engine == 'dask':
        # Can use cumsum as in https://stackoverflow.com/a/60852409/ if it needs
        # to be incrementing, but we just need uniqueness

        def add_id_to_partition(df_part, partition_info=None):
            return (
                df_part
                    .assign(**{col_name: range(len(df_part))})
                    .assign(**{col_name: lambda x: (
                            value_prefix +
                            str(partition_info['number'] if partition_info is not None else 0) +
                            '_' +
                            x[col_name].astype(str)
                        ).astype('large_string[pyarrow]')}
                    )
            )

        df = df.map_partitions(add_id_to_partition)

        return df
    else:
        raise ValueError()

def add_unique_record_id(df, dataset_name):
    return add_unique_id_col(df, col_name='record_id', value_prefix=f'{dataset_name}_')

# Initializes a table listing the pairs between record_ids and source record_ids.
# Should only be called on "source records"; that is, records that
# come directly out of pseudopeople.
def record_id_to_single_source_record_pairs(df, source_col='record_id'):
    if source_col == 'record_id':
        # We can't have duplicate column names, so we make a new column
        # literally called 'source_col'
        df = df.assign(source_col=lambda df: df[source_col])
        source_col = 'source_col'

    return df[['record_id', source_col]].rename(columns={source_col: 'source_record_id'})

In [12]:
# Operations that aggregate records, combining the source record pairs
# between all records that are aggregated

def merge_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix, *args, **kwargs):
    assert len(dfs) == len(source_record_pairings)
    for df in dfs:
        assert 'record_id' in df.columns

    on = kwargs.get('on', None)
    if on is not None:
        # If there are nulls in any of the merge columns, they can't match to anything
        dfs = [df.dropna(subset=on, how='any') for df in dfs]

    result = dfs[0]
    source_record_pairs = source_record_pairings[0]
    dfs_and_source_record_pairs_to_combine = list(zip(dfs[1:], source_record_pairings[1:]))
    for index, (df_to_merge, source_record_pairs_to_merge) in enumerate(dfs_and_source_record_pairs_to_combine):
        result = (
            result.merge(df_to_merge, *args, **kwargs)
        )
        if index == len(dfs_and_source_record_pairs_to_combine) - 1:
            # Since this is the last step, these are the record_ids that will actually be returned
            accumulate_step_record_id_prefix = new_record_id_prefix
        else:
            # A dummy intermediate -- this shouldn't be exposed to the user
            accumulate_step_record_id_prefix = f'merge_iter_{index}'

        result = add_unique_record_id(result, accumulate_step_record_id_prefix)
        source_record_pairs = concat_avoid_fragmentation([
            # The pairs that were already in result
            source_record_pairs
                .rename(columns={'record_id': 'record_id_x'})
                .merge(result[['record_id', 'record_id_x']], on='record_id_x')
                .drop(columns=['record_id_x']),
            # The new ones
            source_record_pairs_to_merge
                .rename(columns={'record_id': 'record_id_y'})
                .merge(result[['record_id', 'record_id_y']], on='record_id_y')
                .drop(columns=['record_id_y']),
        ])
        result = result.drop(columns=['record_id_x', 'record_id_y'])

    return result, source_record_pairs


def dedupe_preserving_source_records(df, source_record_pairs, columns_to_dedupe, new_record_id_prefix):#, source_records_col='source_record_ids'):
    result = drop_duplicates(df[columns_to_dedupe])
    result = add_unique_record_id(result, new_record_id_prefix)
    df_to_result_mapping = (
        df[['record_id'] + columns_to_dedupe]
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(result, on=columns_to_dedupe)
            [['record_id', 'record_id_pre_dedupe']]
    )
    result_source_record_pairs = (
        source_record_pairs
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(df_to_result_mapping, on='record_id_pre_dedupe')
            .drop(columns=['record_id_pre_dedupe'])
    )
    return result, result_source_record_pairs


def concat_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix):
    dfs = [df.rename(columns={'record_id': 'record_id_pre_concat'}) for df in dfs]
    result = concat_avoid_fragmentation(dfs, ignore_index=True)
    result = add_unique_record_id(result, new_record_id_prefix)

    record_id_mapping = (
        result[['record_id', 'record_id_pre_concat']]
    )
    validate_kwarg = {
        'validate': 'm:1',
    } if compute_engine != 'dask' else {} # Dask doesn't support validate

    all_source_record_pairings = persist(concat_avoid_fragmentation(source_record_pairings, ignore_index=False))
    result_source_record_pairs = persist(
        all_source_record_pairings
            .rename(columns={'record_id': 'record_id_pre_concat'})
            .merge(record_id_mapping, on='record_id_pre_concat', **validate_kwarg)
            .drop(columns=['record_id_pre_concat'])
    )

    if compute_engine == 'dask':
        # Manual alternative to the 'validate' kwarg
        assert len(result_source_record_pairs) == len(all_source_record_pairings)

    return result.drop(columns=['record_id_pre_concat']), result_source_record_pairs

### Simulated SSA Numident

In [13]:
%%time

simulated_ssa_numident = pd.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_ssa_numident.parquet')
simulated_ssa_numident = increase_string_capacity(simulated_ssa_numident)
simulated_ssa_numident = add_unique_record_id(simulated_ssa_numident, 'simulated_ssa_numident')
simulated_ssa_numident = persist(simulated_ssa_numident)
simulated_ssa_numident_source_record_pairs = persist(record_id_to_single_source_record_pairs(simulated_ssa_numident))

CPU times: user 102 ms, sys: 42.5 ms, total: 144 ms
Wall time: 132 ms


In [14]:
simulated_ssa_numident_ground_truth = simulated_ssa_numident[['record_id', 'simulant_id']]
simulated_ssa_numident = simulated_ssa_numident.drop(columns=['simulant_id'])
simulated_ssa_numident, simulated_ssa_numident_ground_truth = persist(simulated_ssa_numident, simulated_ssa_numident_ground_truth)
simulated_ssa_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date
0,simulated_ssa_numident_0,786-77-6454,Evelyn,Nancy,Hines,19191204,Female,creation,19191204
1,simulated_ssa_numident_1,688-88-6377,George,Robert,Dickens,19210616,Male,creation,19210616
2,simulated_ssa_numident_2,651-33-9561,Beatrice,Jennie,Fackler,19220113,Female,creation,19220113
3,simulated_ssa_numident_3,665-25-7858,Eura,Nadine,Crusen,19220305,Female,creation,19220305
4,simulated_ssa_numident_4,875-10-2359,Roberta,Ruth,Muilenburg,19220306,Female,creation,19220306
...,...,...,...,...,...,...,...,...,...
19866,simulated_ssa_numident_19866,203-10-6855,Birdie,Lizzy,Mercer,None,Female,creation,20291223
19867,simulated_ssa_numident_19867,505-88-8814,Levi,Holden,Smal,None,Male,creation,20291224
19868,simulated_ssa_numident_19868,417-18-3811,Alan,Messiah,Bazan,20291227,Male,creation,20291227
19869,simulated_ssa_numident_19869,450-74-3175,Emmi,Everlynn,H,20291228,Female,creation,20291228


### 1040 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files -- see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [15]:
tax_years = list(range(2025, 2030))
tax_years

[2025, 2026, 2027, 2028, 2029]

In [16]:
%%time

# Combine 1040 for all years.
simulated_taxes_1040 = concat_avoid_fragmentation([
    pd.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_1040_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)
simulated_taxes_1040 = add_unique_record_id(simulated_taxes_1040, 'simulated_1040')
simulated_taxes_1040 = increase_string_capacity(simulated_taxes_1040)
simulated_taxes_1040 = persist(simulated_taxes_1040)
simulated_taxes_1040_source_record_pairs = persist(record_id_to_single_source_record_pairs(simulated_taxes_1040))

CPU times: user 296 ms, sys: 36.1 ms, total: 332 ms
Wall time: 245 ms


In [17]:
simulated_taxes_1040_ground_truth = simulated_taxes_1040[['record_id', 'simulant_id']]
simulated_taxes_1040 = simulated_taxes_1040.drop(columns=['simulant_id'])
simulated_taxes_1040, simulated_taxes_1040_ground_truth = persist(simulated_taxes_1040, simulated_taxes_1040_ground_truth)
simulated_taxes_1040

,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_1040_0,794-23-1522,Heather,J,Campusano Robledo,None,None,None,14011,Anytown,WA,00000
1,simulated_1040_1,004-53-9627,Bailey,J,Nicholas,1934,cypress ave,None,None,Anytown,WA,00000
2,simulated_1040_2,273-52-7750,Heather,S,Blackwell,57,shadowood dr,None,None,Anytown,WA,00000
3,simulated_1040_3,384-45-8523,Candy,S,Luedeman,10881,maple av,None,None,Anytown,WA,00000
4,simulated_1040_4,858-84-4517,Patrick,J,Morfin,103,71st st w,None,None,Anytown,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...
39114,simulated_1040_39114,595-04-6651,Akari,L,Parkinson,2921,curtis la,None,None,Anytown,WA,00000
39115,simulated_1040_39115,523-63-9324,Jada,T,Franco Resendez,1920,white ibis ct,None,None,Anytown,WA,00000
39116,simulated_1040_39116,187-01-0690,Bianca,L,Sargent,12613,teppert,None,None,Anytown,WA,00000
39117,simulated_1040_39117,808-04-3639,Ashton,N,Cardenas,116,wakema rd,None,None,Anytown,WA,00000


### W2/1099 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files --
see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [18]:
%%time

# Combine W2/1099 for all years.
simulated_w2_1099 = concat_avoid_fragmentation([
    pd.read_parquet(
        f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_taxes_w2_and_1099_{year}.parquet',
        columns=[
            'simulant_id',
            'ssn',
            'first_name',
            'middle_initial',
            'last_name',
            'mailing_address_street_number',
            'mailing_address_street_name',
            'mailing_address_unit_number',
            'mailing_address_po_box',
            'mailing_address_city',
            'mailing_address_state',
            'mailing_address_zipcode',
        ],
    )
    for year in tax_years
], ignore_index=True)
simulated_w2_1099 = add_unique_record_id(simulated_w2_1099, 'simulated_w2_1099')
simulated_w2_1099 = persist(increase_string_capacity(simulated_w2_1099))
simulated_w2_1099_source_record_pairs = persist(record_id_to_single_source_record_pairs(simulated_w2_1099))

CPU times: user 232 ms, sys: 82.6 ms, total: 315 ms
Wall time: 211 ms


In [19]:
simulated_w2_1099_ground_truth = simulated_w2_1099[['record_id', 'simulant_id']]
simulated_w2_1099 = simulated_w2_1099.drop(columns=['simulant_id'])
simulated_w2_1099, simulated_w2_1099_ground_truth = persist(simulated_w2_1099, simulated_w2_1099_ground_truth)
simulated_w2_1099

,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_w2_1099_0,854-13-6295,Erin,R,No,1501,interlake ave n,None,None,Anytown,WA,00000
1,simulated_w2_1099_1,854-13-6295,Erin,R,Stark Lozano,1501,interlake ave n,None,None,Anytown,WA,00000
2,simulated_w2_1099_2,794-23-1522,Heather,J,Campusano Robledo,None,None,None,14011,Anytown,WA,00000
3,simulated_w2_1099_3,093-85-3512,Melisa,D,Velazquez,927,broomfield ln,None,None,Anytown,WA,00000
4,simulated_w2_1099_4,093-85-3512,Melisa,D,Velazquez,927,broomfield ln,None,None,Anytown,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...
53637,simulated_w2_1099_53637,119-02-0252,Paul,None,Hurt,3515,javello rd,None,None,Anytown,WA,00000
53638,simulated_w2_1099_53638,079-75-0734,Lori,T,Washington,9358,e 58th st,None,None,Anytown,WA,00000
53639,simulated_w2_1099_53639,079-75-0734,Lori,None,Washington,9358,e 58th st,None,None,Anytown,WA,00000
53640,simulated_w2_1099_53640,287-19-2431,Patricia,C,Ehle,None,rdg park dr,None,None,Anytown,WA,00000


In [20]:
simulated_taxes, simulated_taxes_source_record_pairs = concat_preserving_source_records(
    [simulated_taxes_1040, simulated_w2_1099],
    [simulated_taxes_1040_source_record_pairs, simulated_w2_1099_source_record_pairs],
    new_record_id_prefix='simulated_taxes',
)

In [21]:
simulated_taxes_1040_with_itins = simulated_taxes_1040[simulated_taxes_1040.ssn.notnull() & simulated_taxes_1040.ssn.str.startswith('9')]
simulated_taxes_1040_with_itins_source_record_pairs = simulated_taxes_1040_source_record_pairs.merge(simulated_taxes_1040_with_itins[['record_id']], on='record_id', how='inner')
simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs = persist(simulated_taxes_1040_with_itins, simulated_taxes_1040_with_itins_source_record_pairs)

%xdel simulated_taxes_1040
%xdel simulated_taxes_1040_source_record_pairs

In [22]:
%xdel simulated_w2_1099
%xdel simulated_w2_1099_source_record_pairs

In [23]:
# "... many of the [IRS] records contain only the first four letters of the last name."
# (Brown et al. 2023, p.30, footnote 19)
# This should be updated in pseudopeople but for now we do it here.
# Note that this truncation only matters for ITIN PIKing since for SSNs that are present in SSA we use name from SSA.
PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION = 0.4 # is this a good guess at "many" in the quote above?
if compute_engine == 'dask':
    # import dask.array as np
    # rng = np.random.default_rng(seed=1234)
    # random_df = pd.from_dask_array(rng.random(len(simulated_taxes), chunks=int(np.ceil(len(simulated_taxes) / simulated_taxes.npartitions))), columns=['random'])
    # random_df.index = simulated_taxes.index
    # to_truncate = simulated_taxes.assign(random=random_df.random).random < PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION
    to_truncate, not_to_truncate = simulated_taxes.random_split([PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, 1 - PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION], random_state=1234)
    to_truncate = to_truncate.assign(last_name=to_truncate.last_name.str[:4])
    simulated_taxes = concat_avoid_fragmentation([to_truncate, not_to_truncate], ignore_index=True)
else:
    import numpy as np
    to_truncate = simulated_taxes.sample(frac=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, random_state=1234).index

    simulated_taxes.loc[to_truncate, 'last_name'] = simulated_taxes.loc[to_truncate, 'last_name'].str[:4]

In [24]:
simulated_taxes, simulated_taxes_source_record_pairs = persist(simulated_taxes, simulated_taxes_source_record_pairs)

In [25]:
source_record_ground_truth = persist(concat_avoid_fragmentation([
    simulated_ssa_numident_ground_truth,
    simulated_taxes_1040_ground_truth,
    simulated_w2_1099_ground_truth,
]).rename(columns={'record_id': 'source_record_id'}))
source_record_ground_truth

,source_record_id,simulant_id
0,simulated_ssa_numident_0,0_19979
1,simulated_ssa_numident_1,0_6846
2,simulated_ssa_numident_2,0_19983
3,simulated_ssa_numident_3,0_262
4,simulated_ssa_numident_4,0_12473
...,...,...
53637,simulated_w2_1099_53637,0_8910
53638,simulated_w2_1099_53638,0_2378
53639,simulated_w2_1099_53639,0_2378
53640,simulated_w2_1099_53640,0_22909


In [26]:
%xdel simulated_taxes_1040_ground_truth
%xdel simulated_w2_1099_ground_truth

### 2030 Census Unedited File (CUF)

For now, we gloss over the data schema for addresses.
We don't know how addresses would be formatted in the CUF (and it's hard to guess, because
address is not part of the Census form), but it likely would have some of these fields
(street number, street name, etc) combined.

While PVS input files do not in general have names split into first, middle, and last,
I am guessing the CUF **would** have first name, middle initial, last name (which is how pseudopeople
generates it), because that [matches the Census questionnaire](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/questionnaires-and-instructions/questionnaires/2020-informational-questionnaire-english_DI-Q1.pdf).

In [27]:
%%time

simulated_census_2030 = pd.read_parquet(f'{output_dir}/{data_to_use}/pseudopeople_simulated_datasets/simulated_census_2030.parquet')
simulated_census_2030 = add_unique_record_id(simulated_census_2030, 'simulated_census_2030')
simulated_census_2030 = increase_string_capacity(simulated_census_2030)
simulated_census_2030 = persist(simulated_census_2030)

CPU times: user 50.5 ms, sys: 28.8 ms, total: 79.3 ms
Wall time: 44.7 ms


In [28]:
simulated_census_2030_ground_truth = simulated_census_2030[['record_id', 'simulant_id']]
simulated_census_2030 = simulated_census_2030.drop(columns=['simulant_id'])
simulated_census_2030, simulated_census_2030_ground_truth = persist(simulated_census_2030, simulated_census_2030_ground_truth)
simulated_census_2030

,record_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,simulated_census_2030_0,0_8033,Gerald,R,Allen,86,11/03/1943,1130,mallory ln,None,Anytown,WA,00000,Household,Reference person,Male,Black,2030
1,simulated_census_2030_1,0_1066,April,S,Hayden,33,10/23/1996,32597,delacorte dr,None,Anytown,WA,00000,Household,Other nonrelative,Female,Black,2030
2,simulated_census_2030_2,0_1066,Loretta,T,Lowe,71,06/01/1958,32597,delacorte dr,None,Anytown,WA,00000,Household,Reference person,Female,White,2030
3,simulated_census_2030_3,0_2514,Sandra,S,Sorrentino,75,03/18/1954,4458,windsor pl,None,Anytown,WA,00000,Household,Reference person,Female,Multiracial or Other,2030
4,simulated_census_2030_4,0_5627,Bobby,S,Baker,44,05/20/1985,None,winding trail rd,None,Anytown,WA,00000,Household,Other nonrelative,Male,White,2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024,simulated_census_2030_11024,0_10778,Jeremy,T,Boyd,46,07/01/1983,211,quiet way,None,Anytown,WA,00000,Household,Reference person,Male,Black,2030
11025,simulated_census_2030_11025,0_11001,Wendy,M,Gross,54,12/05/1975,2801,blue rdg dr n,None,Anytown,WA,00000,Household,Reference person,Female,White,2030
11026,simulated_census_2030_11026,0_5308,Ember,H,Samuels,10,10/26/2019,24113,lauder,None,Anytown,WA,00000,Household,Reference person,Female,Black,2030
11027,simulated_census_2030_11027,0_10693,Athena,V,Deshpande,27,07/05/2002,1534,bentley dr,None,Anytown,WA,00000,Household,Reference person,Female,Asian,2030


In [29]:
save_file_with_ground_truth('simulated_census_2030', simulated_census_2030, simulated_census_2030_ground_truth)

In [30]:
%xdel simulated_census_2030
%xdel simulated_census_2030_ground_truth

## Create reference files

> The Census Numident – all Social Security Administration (SSA) Numident SSN records are
  edited (collapsed) to produce a Census Numident file that contains “one best-data record” for
  each SSN. All variants of name information for each SSN are retained in the Alternate Name
  Numident file, while all variants of date of birth data are retained in the Alternate DOB
  Numident. The SSN-PIK crosswalk file is used to attach a corresponding unique PIK value for
  each SSN value in the Census Numident file.

### Census Numident

Luque and Wagner, p. 4:
  
> The SSA Numident file contains all transactions ever recorded against any single SSN - with each entry
representing an addition or change (such as name changes) to the SSN record. This file is edited to
create the **Census Numident**, which contains one record for each SSN. Each SSN record in the Census
Numident contains name, DOB, sex, race, place of birth, parents’ name, citizenship status and date of death information.

and in footnote 5:

> Name edits, DOB reconciliation, and race identifiers are some of the edits conducted to produce this Numident
file. **The resulting Numident file contains the most recent name and DOB data.**

We are missing quite a few columns, since they are missing in pseudopeople's SSA Numident: race, place of birth, parents' name,
citizenship status.
However, I'm pretty sure none of these are used in linking.

In [31]:
def fill_dates(df, fill_with):
    return (
        # Replace invalid dates with nans
        pd.to_datetime(df.event_date, format='%Y%m%d', errors='coerce')
            .fillna(pandas.to_datetime('2100-01-01' if fill_with == 'latest' else '1900-01-01'))
    )

def best_data_from_columns(df, columns, best_is_latest=True):
    # We don't want to throw out events with a missing/invalid date, so we'll fill them with the value *least* likely to be chosen
    # (earlier than all values if taking the latest, later than all values if taking the earliest).
    fill_with = 'earliest' if best_is_latest else 'latest'

    result = (
        df
            # Without mutating the existing date column, get one that is actually
            # a date type and can be used for sorting.
            # Note: we actually convert this to an integer for sorting purposes, because Modin was having trouble
            # sorting by it as an actual datetime
            .assign(event_date_for_sort=lambda df: fill_dates(df, fill_with=fill_with).astype(numpy.int64) // 10 ** 9)
            .dropna(subset=columns, how='all')
            .pipe(lambda df: drop_duplicates(df, subset='ssn', sort_col='event_date_for_sort', keep=('last' if best_is_latest else 'first')))
            [['record_id', 'ssn'] + columns]
    )

    return result, record_id_to_single_source_record_pairs(result)

best_name, best_name_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['first_name', 'middle_name', 'last_name'],
)

best_date_of_birth, best_date_of_birth_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident,
    columns=['date_of_birth'],
)

best_date_of_death, best_date_of_death_source_record_pairs = best_data_from_columns(
    simulated_ssa_numident[simulated_ssa_numident.event_type == 'death'],
    columns=['event_date'],
)
best_date_of_death = best_date_of_death.rename(columns={'event_date': 'date_of_death'})

simulated_census_numident, simulated_census_numident_source_record_pairs = persist(merge_preserving_source_records(
    [best_name, best_date_of_birth, best_date_of_death],
    [best_name_source_record_pairs, best_date_of_birth_source_record_pairs, best_date_of_death_source_record_pairs],
    new_record_id_prefix='simulated_census_numident',
    on=['ssn'],
    how='left',
))
simulated_census_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,date_of_death
0,simulated_census_numident_0,339-86-1863,Aaron,Jeffrey,Scowcroft,19781223,NaN
1,simulated_census_numident_1,890-88-5417,Jenna,Kelly,Cruz,19921015,NaN
2,simulated_census_numident_2,567-62-4426,Samuel,Tanner,Brooks,19960824,NaN
3,simulated_census_numident_3,261-64-8287,Ashley,Elise,Robertson,19900221,NaN
4,simulated_census_numident_4,044-64-0926,Scott,Joseph,Isaacson,19710808,NaN
...,...,...,...,...,...,...,...
18464,simulated_census_numident_18464,538-79-0978,Lucas,Jacob,Shumpert,20131215,NaN
18465,simulated_census_numident_18465,259-20-6171,Chayse,Braylon,Zellers,20161021,NaN
18466,simulated_census_numident_18466,420-51-1362,Bruce,Alan,Patel-Capshaw,19630414,NaN
18467,simulated_census_numident_18467,316-98-8999,Jonathan,Benjamin,Cando Chenery,20000923,22000922


### Alternate Name Numident

Wagner and Layne, p. 9:

>  All variants of name information for each SSN are retained in the Alternate Name
Numident file...

In [32]:
simulated_alternate_name_numident, simulated_alternate_name_numident_source_record_pairs = persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_alternate_name_numident',
))
simulated_alternate_name_numident

,record_id,ssn,first_name,middle_name,last_name
0,simulated_alternate_name_numident_11,685-77-0916,Betty,Audrey,Keel
1,simulated_alternate_name_numident_21,365-44-3027,Betty,Mary,None
2,simulated_alternate_name_numident_22,512-15-1742,Martha,Beulah,Burandt
3,simulated_alternate_name_numident_25,590-86-8079,Elizabeth,Ora,Pufveah
4,simulated_alternate_name_numident_27,101-33-5970,Betty,Deva,Watts
...,...,...,...,...,...
18744,simulated_alternate_name_numident_19866,203-10-6855,Birdie,Lizzy,Mercer
18745,simulated_alternate_name_numident_19867,505-88-8814,Levi,Holden,Smal
18746,simulated_alternate_name_numident_19868,417-18-3811,Alan,Messiah,Bazan
18747,simulated_alternate_name_numident_19869,450-74-3175,Emmi,Everlynn,H


In [33]:
compute(groupby_agg_small_groups(simulated_alternate_name_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    18469.000000
mean         1.015161
std          0.122195
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [34]:
# Show alternate names for the same simulated SSN
simulated_alternate_name_numident.merge(
    groupby_agg_small_groups(simulated_alternate_name_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,record_id,ssn,first_name,middle_name,last_name
453,simulated_alternate_name_numident_19750,752-57-7892,Edward,Kirk,Jasaitij
452,simulated_alternate_name_numident_3516,752-57-7892,Edward,Kirk,Jasaitis
232,simulated_alternate_name_numident_906,757-47-3233,Fred,Ronald,Hill
233,simulated_alternate_name_numident_16483,757-47-3233,Fred,Man,Hill
230,simulated_alternate_name_numident_892,764-34-9283,Norma,Annoe,Nelson
...,...,...,...,...,...
146,simulated_alternate_name_numident_428,896-40-7512,Ronald,Walter,Sciulli
228,simulated_alternate_name_numident_883,897-75-8780,None,Barbara,Gregory
229,simulated_alternate_name_numident_19718,897-75-8780,Mona,Barbara,Gregory
357,simulated_alternate_name_numident_18365,899-24-9400,Willie,David,Laduke


### Alternate DOB Numident

Wagner and Layne, p. 9:

> ... while all variants of date of birth data are retained in the Alternate DOB
Numident.

In [35]:
simulated_alternate_dob_numident, simulated_alternate_dob_numident_source_record_pairs = persist(dedupe_preserving_source_records(
    simulated_ssa_numident,
    simulated_ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'date_of_birth'],
    new_record_id_prefix='simulated_alternate_dob_numident',
))
simulated_alternate_dob_numident

,record_id,ssn,date_of_birth
0,simulated_alternate_dob_numident_0,786-77-6454,19191204
1,simulated_alternate_dob_numident_2,651-33-9561,19220113
2,simulated_alternate_dob_numident_14,765-44-4521,None
3,simulated_alternate_dob_numident_34,027-46-6956,19240918
4,simulated_alternate_dob_numident_48,819-18-8122,19256730
...,...,...,...
18596,simulated_alternate_dob_numident_19866,203-10-6855,None
18597,simulated_alternate_dob_numident_19867,505-88-8814,None
18598,simulated_alternate_dob_numident_19868,417-18-3811,20291227
18599,simulated_alternate_dob_numident_19869,450-74-3175,20291228


In [36]:
compute(groupby_agg_small_groups(simulated_alternate_dob_numident, by='ssn', agg_func=lambda x: x.size()).describe())

count    18469.000000
mean         1.007147
std          0.084240
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [37]:
simulated_alternate_dob_numident.merge(
    groupby_agg_small_groups(simulated_alternate_dob_numident[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner'
).sort_values('ssn').tail(100)

,record_id,ssn,date_of_birth
23,simulated_alternate_dob_numident_16273,620-64-0370,None
22,simulated_alternate_dob_numident_120,620-64-0370,19300508
101,simulated_alternate_dob_numident_17021,621-71-8672,19430325
100,simulated_alternate_dob_numident_825,621-71-8672,None
29,simulated_alternate_dob_numident_19395,626-48-7965,19521222
...,...,...,...
11,simulated_alternate_dob_numident_16404,876-22-5160,19261111
70,simulated_alternate_dob_numident_445,876-50-3136,19380720
71,simulated_alternate_dob_numident_19714,876-50-3136,19380721
194,simulated_alternate_dob_numident_2971,899-07-0109,19650509


### Name/DOB Reference File

Wagner and Layne, p. 9:

> The Name and DOB Reference files are reformatted versions of the Census Numident
and includes **all possible combinations of alternate names and dates of birth, as well as
ITIN data**. All of the reference files contain SSN/ITIN and the corresponding PIK. When
an input record is linked to a reference file, the corresponding PIK is assigned. Table 1
presents the number of observations in each of the reference files.

A slightly confusing point: sometimes the Name and DOB reference files are described
as one and the same thing, and sometimes as separate.
I believe this is because **they differ only in how they are "cut" for the PVS process:**
the name reference file is cut by first and last initial,
while the DOB reference file is cut by month and day of birth.

This is described in Wagner and Layne, p.15:

> The [DOBSearch] module matches against a re-split
version of the Numident Name Reference file, splitting the data based on month and day
of birth.

Since we handle the logic of "cutting" in the linkage process itself, we generate
a single reference file here.

Note that unlike for addresses, and unlike for the pre-processing of PVS *input* files
(as opposed to reference files), there is no explicit nickname processing/correction here.
I am fairly sure that is accurate to the real PVS, which I believe assumes that nicknames
would not be present in SSA/tax records (or at least, that the real name would appear
at least once in these records).

In [38]:
simulated_name_dob_numident_records, simulated_name_dob_numident_records_source_record_pairs = persist(merge_preserving_source_records(
    [simulated_alternate_name_numident, simulated_alternate_dob_numident],
    [simulated_alternate_name_numident_source_record_pairs, simulated_alternate_dob_numident_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='name_dob_numident_records',
))
simulated_name_dob_numident_records

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,name_dob_numident_records_0,685-77-0916,Betty,Audrey,Keel,19221205
1,name_dob_numident_records_1,365-44-3027,Betty,Mary,None,19230809
2,name_dob_numident_records_2,512-15-1742,Martha,Beulah,Burandt,19230907
3,name_dob_numident_records_3,590-86-8079,Elizabeth,Ora,Pufveah,19231218
4,name_dob_numident_records_4,101-33-5970,Betty,Deva,Watts,19240505
...,...,...,...,...,...,...
18904,name_dob_numident_records_18904,203-10-6855,Birdie,Lizzy,Mercer,None
18905,name_dob_numident_records_18905,505-88-8814,Levi,Holden,Smal,None
18906,name_dob_numident_records_18906,417-18-3811,Alan,Messiah,Bazan,20291227
18907,name_dob_numident_records_18907,450-74-3175,Emmi,Everlynn,H,20291228


In [39]:
# Show alternate name/DOB for the same simulated SSN
simulated_name_dob_numident_records.merge(
    groupby_agg_small_groups(simulated_name_dob_numident_records[["ssn"]], by="ssn", agg_func=lambda x: x.size()).pipe(lambda x: x[x > 1]).reset_index()[['ssn']],
    on='ssn',
    how='inner',
).sort_values('ssn').tail(100)

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
345,name_dob_numident_records_16557,790-98-9882,None,Daniel,Nemecek,19500721
344,name_dob_numident_records_1074,790-98-9882,Curtis,Daniel,Nemecek,19500721
439,name_dob_numident_records_16199,791-62-7882,Rick,Douglas,Weston,19570312
438,name_dob_numident_records_1940,791-62-7882,Eric,Douglas,Weston,19570312
343,name_dob_numident_records_17156,793-06-7496,Nancy,Mary,Martin,19500422
...,...,...,...,...,...,...
255,name_dob_numident_records_18745,897-75-8780,Mona,Barbara,Gregory,19431220
705,name_dob_numident_records_16717,899-07-0109,Jeffrey,Glenn,Hawkins,19580911
704,name_dob_numident_records_16716,899-07-0109,Jeffrey,Glenn,Hawkins,19650509
401,name_dob_numident_records_17344,899-24-9400,Willie,David,Laduke,19530914


#### Incorporating people with ITINs

Individual Taxpayer Identification Numbers (ITINs) can be issued to people who are required to file
federal taxes but are not eligible for a Social Security Number.
The most common reason for this is being an undocumented immigrant and therefore not being authorized
to work in the United States.

People without SSNs used to be impossible to assign PIKs to.
In 2011 the NORC report stated (p. 38, footnote 19):

> NORC understands that the Census Bureau has undertaken an effort to enhance the PVS reference files with IRS
files that include Individual Taxpayer Identification Numbers (ITIN). For those people who are required to file a tax
return but do not have, and may not want an SSN—such as a non-U.S. citizen—the IRS issues the taxpayer an ITIN.
This enhancement to the PVS reference file may help to match more non-U.S citizens.

By 2014 (Wagner and Layne, p. 5):

> One of the key enhancements [made in recent years] increased the coverage of the reference files by
including records for persons with Individual Taxpayer Identification Numbers assigned
by the Internal Revenue Service (ITINs) to [along with?] the SSN-based Numident data. 

I have not found a specific description of how ITIN records are constructed in any of the
publicly-available sources.
This may be because it is straightforward, or because the tax data schema is confidential.
I assume that only IRS data is used, since no other data source that I am aware of would
report ITIN.

It is stated that the ITIN records are created directly from tax filings and not
from ITIN applications (Brown et al. p. 29, footnote 16), which is convenient
because the tax filing data is what we can simulate with
pseudopeople:

> The NUMIDENT provides the PII on the SSN-holder from the issuing agency (SSA), and that PII is used in SSN
verification. **For ITINs, the Census Bureau does not have access to the ITIN applications** to the issuing agency (IRS),
so name and DOB verification of ITINs is less reliable.

"Less reliable" is a bit confusing here, because as stated above when generating
the simulated tax data, IRS data should not contain date of birth at all.
Here, we have stayed true to this by omitting it entirely.

We assume that only 1040 filings would be used for this purpose; we wouldn't expect ITINs to
show up on employer-filed W-2/1099 forms.

In [40]:
# Analogous to the process of getting alternate names and dates of birth
# from SSA, we retain all versions of the name from taxes.
simulated_name_for_itins, simulated_name_for_itins_source_record_pairs = persist(dedupe_preserving_source_records(
    simulated_taxes_1040_with_itins.rename(columns={'middle_initial': 'middle_name'}),
    simulated_taxes_1040_with_itins_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='simulated_name_for_itins',
))
simulated_name_for_itins

,record_id,ssn,first_name,middle_name,last_name
0,simulated_name_for_itins_93,905-55-4311,Pate,R,Brown-Villarreal
1,simulated_name_for_itins_112,970-57-8403,Arthur,M,None
2,simulated_name_for_itins_133,978-95-3331,Stephen,P,Occupant
3,simulated_name_for_itins_378,917-62-7164,Susie,B,Thomas
4,simulated_name_for_itins_506,921-53-9692,Toni,L,Andreadiz
...,...,...,...,...,...
603,simulated_name_for_itins_39032,977-84-6791,Liam,L,Wheatley
604,simulated_name_for_itins_39075,997-95-0405,Joseph,D,Dellinger
605,simulated_name_for_itins_39080,939-53-1702,Josiah,L,Mcmiller
606,simulated_name_for_itins_39086,993-56-2286,Larisa,A,Beilfuss


In [41]:
compute(groupby_agg_small_groups(simulated_name_for_itins, by='ssn', agg_func=lambda x: x.size()).describe())

count    510.000000
mean       1.192157
std        0.432402
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        4.000000
dtype: float64

In [42]:
# How many records in the simulated name/DOB numident are in the ITIN range?
# With normal pseudopeople settings, this shouldn't happen, because SSN in SSA is noiseless.
# If this number is >0, we may be adding ITIN records as more alternates to existing records.
compute(simulated_name_dob_numident_records.ssn.str.startswith('9').sum())

0

In [43]:
simulated_name_dob_reference_file, simulated_name_dob_reference_file_source_record_pairs = persist(concat_preserving_source_records(
    [simulated_name_dob_numident_records, simulated_name_for_itins],
    [simulated_name_dob_numident_records_source_record_pairs, simulated_name_for_itins_source_record_pairs],
    new_record_id_prefix='simulated_name_dob_reference_file',
))
simulated_name_dob_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,simulated_name_dob_reference_file_0,685-77-0916,Betty,Audrey,Keel,19221205
1,simulated_name_dob_reference_file_1,365-44-3027,Betty,Mary,None,19230809
2,simulated_name_dob_reference_file_2,512-15-1742,Martha,Beulah,Burandt,19230907
3,simulated_name_dob_reference_file_3,590-86-8079,Elizabeth,Ora,Pufveah,19231218
4,simulated_name_dob_reference_file_4,101-33-5970,Betty,Deva,Watts,19240505
...,...,...,...,...,...,...
19512,simulated_name_dob_reference_file_19512,977-84-6791,Liam,L,Wheatley,NaN
19513,simulated_name_dob_reference_file_19513,997-95-0405,Joseph,D,Dellinger,NaN
19514,simulated_name_dob_reference_file_19514,939-53-1702,Josiah,L,Mcmiller,NaN
19515,simulated_name_dob_reference_file_19515,993-56-2286,Larisa,A,Beilfuss,NaN


### GeoBase Reference File

Wagner and Layne, p. 9:

> PVS creates three other sets of reference
files containing Numident data: the **GeoBase Reference File**, the Name Reference File,
and the DOB Reference file.
The GeoBase Reference File appends addresses from administrative records attached
to Numident data, including all possible combinations of alternate names and dates of
birth for SSN. Addresses from administrative records are edited and processed through
commercial software product to clean and standardize address data. ITIN data is also
incorporated into the Geobase.

Luque and Wagner, p. 5:

> Reference files contain data from the Numident file enhanced with address
data obtained from federal AR [administrative records] files.<sup>8</sup>
The reference files, thus, contain all variants of a person’s name, DOB,
and sex, as well as current and recent addresses. These reference files are
referred to as the (PVS) Geobase reference file since addresses (a geographic component)
are appended to each person record.<sup>9</sup> It is important to note that there are
multiple Geobase reference files that are created depending on the vintage of the
incoming file to be processed through PVS.

> <sup>8</sup> Namely, data from the IRS, Department of Housing and Urban Development,
several files from the Department of Health and Human Services, and Selective Service.

> <sup>9</sup> In particular, the address data is cleaned and standardized and used
to construct a variable called GEOKEY. The GEOKEY variable is constructed as a subset
of the full address, and then is appended to the Numident data to create the
PVS Geobase Reference file.

We only have IRS data to use for addresses, and specifically only W-2/1099 data,
which is a limitation of this case study.
I can't find a concrete definition of "recent" -- as noted above, we use 5 years
of IRS data.
This may be longer than the true window, but this may end up making up for
the lack of non-IRS data sources, and get us closer to a realistic number of
alternate addresses.

Also, our address data comes out of pseudopeople already parsed into address parts
like street name, etc.
For more realism, pseudopeople should output a single string that we have to (imperfectly) parse apart.

I haven't been able to find out more about what kind of "subset" the geokey is.
It is unclear to me why geokey is "interesting" since it is just derived from the
address parts.

In [44]:
address_cols = [c for c in simulated_taxes.columns if 'mailing_address' in c]

def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.replace('\s+', ' ', regex=True)
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted above
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', numpy.nan)
    )

simulated_tax_addresses = (
    simulated_taxes
        # Can only link these to the other files if they have an SSN
        .dropna(subset=['ssn'])
        [['record_id', 'ssn'] + address_cols]
        .assign(**{c: lambda df, c=c: standardize_address_part(df[c]) for c in address_cols})
        .pipe(increase_string_capacity)
)

simulated_addresses_by_ssn, simulated_addresses_by_ssn_source_record_pairs = persist(dedupe_preserving_source_records(
    simulated_tax_addresses,
    simulated_taxes_source_record_pairs,
    columns_to_dedupe=['ssn'] + address_cols,
    new_record_id_prefix='addresses_by_ssn',
))
simulated_addresses_by_ssn

,record_id,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,addresses_by_ssn_10,172-15-1292,9808,111 AGEHUE,None,None,ANYTOWN,WA,00000
1,addresses_by_ssn_16,19z-24-265G,None,None,None,6846,ANYTOWN,WA,00000
2,addresses_by_ssn_19,606-92-2881,2327,N COLLEGE AFE,None,None,ANYTOWN,WA,00000
3,addresses_by_ssn_21,678-32-5290,None,E CALHOUN ST,None,None,ANYTOWN,WA,00000
4,addresses_by_ssn_25,327-51-5264,9706,N AVONDALE DR,None,None,ANYTOWN,WA,00000
...,...,...,...,...,...,...,...,...,...
27963,addresses_by_ssn_92754,295-72-7873,116,WAKEMA RD,None,None,ANYTOWN,WA,00000
27964,addresses_by_ssn_92755,518-45-2600,116,WAKEMA RD,None,None,ANYTOWN,WA,None
27965,addresses_by_ssn_92756,119-02-0252,3515,JAVELLO RD,None,None,ANYTOWN,WA,00000
27966,addresses_by_ssn_92758,079-75-0734,9358,E 58TH ST,None,None,ANYTOWN,WA,00000


In [45]:
num_addresses = groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).rename('size').reset_index().sort_values('size')
num_addresses

,ssn,size
16364,Z80-15-7958,1
16365,n3-62-3143,1
16366,o11-81-119Z,1
16367,o47-45-3140,1
16368,o78-53-4232,1
...,...,...
12877,734-84-5703,7
593,034-83-5590,7
12844,732-98-7340,7
12163,696-02-3982,7


In [46]:
# Show some SSNs with a lot of address variation
compute(pd.merge(num_addresses.tail(10), simulated_addresses_by_ssn, on='ssn', how='inner').sort_values('ssn'))

,ssn,size,record_id,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
40,034-83-5590,7,addresses_by_ssn_18016,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,CO,00000
42,034-83-5590,7,addresses_by_ssn_43567,53,S GRAMERCY PL,None,None,ANYTOWN,WA,00000
43,034-83-5590,7,addresses_by_ssn_59105,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,IN,00000
44,034-83-5590,7,addresses_by_ssn_69913,11818,ARBOR MEADOW DRIVE,None,None,None,WA,00000
45,034-83-5590,7,addresses_by_ssn_80642,11818,ARBOR MEADOW DRIVE,None,None,ANYTOWN,WA,00000
...,...,...,...,...,...,...,...,...,...,...
10,737-45-5790,6,addresses_by_ssn_91543,None,N SUNRISE DR,None,None,4NYTOWN,WA,00000
9,737-45-5790,6,addresses_by_ssn_80691,None,N SUNRISE DR,None,None,ANYTOWN,WA,None
8,737-45-5790,6,addresses_by_ssn_34055,None,N SUNRISE DR,None,None,ANYTOWN,WA,01000
7,737-45-5790,6,addresses_by_ssn_25975,None,N SUNRISE DR,None,None,ANYTOWN,WA,00009


In [47]:
# Rough estimate of how many rows we should have in our reference file, once we do this Cartesian product
compute(
    len(simulated_name_dob_reference_file) *
    groupby_agg_small_groups(simulated_addresses_by_ssn, by='ssn', agg_func=lambda x: x.size()).mean()
)

33316.12890625

In [48]:
simulated_geobase_reference_file, simulated_geobase_reference_file_source_record_pairs = persist(merge_preserving_source_records(
    [simulated_name_dob_reference_file, simulated_addresses_by_ssn],
    [simulated_name_dob_reference_file_source_record_pairs, simulated_addresses_by_ssn_source_record_pairs],
    on=['ssn'],
    how='left',
    new_record_id_prefix='simulated_geobase_reference_file',
))
simulated_geobase_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,simulated_geobase_reference_file_0,685-77-0916,Betty,Audrey,Keel,19221205,47461,W ROXBURY DR,None,None,ANYTOWN,WA,00000
1,simulated_geobase_reference_file_1,365-44-3027,Betty,Mary,None,19230809,19236,W LACROSSE STREET,None,None,ANYTOWN,WA,00000
2,simulated_geobase_reference_file_2,512-15-1742,Martha,Beulah,Burandt,19230907,2333,WESTMINSTER PLACE,None,None,ANYTOWN,WA,00000
3,simulated_geobase_reference_file_3,590-86-8079,Elizabeth,Ora,Pufveah,19231218,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,simulated_geobase_reference_file_4,101-33-5970,Betty,Deva,Watts,19240505,4005,ARBOLADO WY,None,None,ANYTOWN,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31343,simulated_geobase_reference_file_31343,939-53-1702,Josiah,L,Mcmiller,NaN,2904,YACHT CLUB PT,None,None,ANYTOWN,WA,00000
31344,simulated_geobase_reference_file_31344,939-53-1702,Josiah,L,Mcmiller,NaN,W14066,E VALENCIA RD,None,None,ANYTOWN,WA,00004
31345,simulated_geobase_reference_file_31345,939-53-1702,Josiah,L,Mcmiller,NaN,W14066,E VALENCIA RD,None,None,ANYTOWN,WA,00000
31346,simulated_geobase_reference_file_31346,993-56-2286,Larisa,A,Beilfuss,NaN,1411,W 80TH ST,None,None,ANYTOWN,WA,00000


In [49]:
# Actual number of rows
len(simulated_geobase_reference_file)

31348

## Track ground truth for reference files

In [50]:
def get_simulants_of_source_records(source_record_pairs, filter_record_ids=None):
    if filter_record_ids is not None:
        source_record_pairs = source_record_pairs.pipe(filter_record_ids)

    result = (
        drop_duplicates(source_record_pairs)
            .merge(source_record_ground_truth, on='source_record_id')
            [['record_id', 'simulant_id']]
            .pipe(drop_duplicates)
    )
    result = result.merge(
        groupby_agg_small_groups(result, by='record_id', agg_func=lambda x: x.simulant_id.nunique().rename('n_unique_simulants')).reset_index(),
        on='record_id',
        how='left',
    )
    return result

def get_ground_truth_for_records(source_record_pairs):
    result = persist(get_simulants_of_source_records(source_record_pairs))

    result_nunique_describe = compute(result['n_unique_simulants'].describe())

    if numpy.isclose(result_nunique_describe.loc['max'], 1):
        print('No collisions')
        return result

    print('Collisions:')
    display(result_nunique_describe)

    print('Simulated tax records with the most collisions:')
    most_collisions_record_id = result.sort_values('n_unique_simulants', ascending=False).head(1).record_id.iloc[0]
    most_collisions_source_record_ids = persist(
        drop_duplicates(source_record_pairs[source_record_pairs.record_id == most_collisions_record_id][['source_record_id']])
    )
    most_collisions_tax_filings = (
        most_collisions_source_record_ids
            .merge(drop_duplicates(simulated_taxes_source_record_pairs[['record_id', 'source_record_id']]), on='source_record_id', how='inner')
            .merge(simulated_taxes, on='record_id', how='left')
            .merge(source_record_ground_truth, on='source_record_id', how='left')
    )
    display(compute(most_collisions_tax_filings))

    print('Simulated SSA records with the most collisions:')
    display(
        compute(
            most_collisions_source_record_ids
                .merge(simulated_ssa_numident.rename(columns={'record_id': 'source_record_id'}), on='source_record_id', how='inner')
                .merge(source_record_ground_truth, on='source_record_id', how='left')
        )
    )

    return result

### Simulated Census Numident

In [51]:
len(simulated_census_numident_source_record_pairs)

38271

In [52]:
simulated_census_numident_ground_truth = get_ground_truth_for_records(simulated_census_numident_source_record_pairs)

No collisions


In [53]:
simulated_census_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_census_numident_0,0_12403,1
1,simulated_census_numident_1,0_3262,1
2,simulated_census_numident_2,0_1128,1
3,simulated_census_numident_3,0_18839,1
4,simulated_census_numident_4,0_14177,1
...,...,...,...
18464,simulated_census_numident_18443,0_19848,1
18465,simulated_census_numident_18436,0_6475,1
18466,simulated_census_numident_18431,0_950,1
18467,simulated_census_numident_18461,0_7242,1


### Simulated Alternate Name Numident

In [54]:
len(simulated_alternate_name_numident_source_record_pairs)

19871

In [55]:
simulated_alternate_name_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_name_numident_source_record_pairs)

No collisions


In [56]:
simulated_alternate_name_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_alternate_name_numident_11,0_2471,1
1,simulated_alternate_name_numident_21,0_17500,1
2,simulated_alternate_name_numident_22,0_19772,1
3,simulated_alternate_name_numident_25,0_19848,1
4,simulated_alternate_name_numident_27,0_3746,1
...,...,...,...
18744,simulated_alternate_name_numident_19866,0_23602,1
18745,simulated_alternate_name_numident_19867,0_23588,1
18746,simulated_alternate_name_numident_19868,0_23589,1
18747,simulated_alternate_name_numident_19869,0_23596,1


### Alternate DOB Numident

In [57]:
len(simulated_alternate_dob_numident_source_record_pairs)

19871

In [58]:
simulated_alternate_dob_numident_ground_truth = get_ground_truth_for_records(simulated_alternate_dob_numident_source_record_pairs)

No collisions


In [59]:
simulated_alternate_dob_numident_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_alternate_dob_numident_0,0_19979,1
1,simulated_alternate_dob_numident_2,0_19983,1
2,simulated_alternate_dob_numident_14,0_1366,1
3,simulated_alternate_dob_numident_34,0_12990,1
4,simulated_alternate_dob_numident_48,0_19944,1
...,...,...,...
18596,simulated_alternate_dob_numident_19866,0_23602,1
18597,simulated_alternate_dob_numident_19867,0_23588,1
18598,simulated_alternate_dob_numident_19868,0_23589,1
18599,simulated_alternate_dob_numident_19869,0_23596,1


### Simulated Name/DOB Reference File

In [60]:
len(simulated_name_dob_reference_file_source_record_pairs)

41937

In [61]:
simulated_name_dob_reference_file_ground_truth = get_ground_truth_for_records(simulated_name_dob_reference_file_source_record_pairs)

No collisions


In [62]:
simulated_name_dob_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_name_dob_reference_file_15594,0_19979,1
1,simulated_name_dob_reference_file_15595,0_19979,1
2,simulated_name_dob_reference_file_17214,0_6846,1
3,simulated_name_dob_reference_file_15049,0_19983,1
4,simulated_name_dob_reference_file_15050,0_19983,1
...,...,...,...
19512,simulated_name_dob_reference_file_19500,0_23319,1
19513,simulated_name_dob_reference_file_19504,0_1328,1
19514,simulated_name_dob_reference_file_19506,0_23720,1
19515,simulated_name_dob_reference_file_19510,0_5904,1


### Simulated GeoBase Reference File

In [63]:
len(simulated_geobase_reference_file_source_record_pairs)

158878

In [64]:
simulated_geobase_reference_file_ground_truth = get_ground_truth_for_records(simulated_geobase_reference_file_source_record_pairs)

Collisions:


count    32688.000000
mean         1.084618
std          0.292153
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          3.000000
Name: n_unique_simulants, dtype: float64

Simulated tax records with the most collisions:


,source_record_id,record_id,ssn,first_name,middle_initial,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,simulant_id
0,simulated_1040_14642,simulated_taxes_14642,324-30-7438,Anthony,K,Lewi,2705,shattalon dr,None,None,Anytown,WA,00000,0_12634
1,simulated_1040_18005,simulated_taxes_18005,324-30-7438,Anthony,K,Lewi,2705,shattalon dr,None,None,Anytown,WA,00000,0_12634
2,simulated_1040_25933,simulated_taxes_25933,324-30-7438,Anthony,K,Lewis,2705,shattalon dr,None,None,Anytown,WA,00000,0_12634
3,simulated_1040_34008,simulated_taxes_34008,324-30-7438,Anthony,K,Occupant,2705,shattalon dr,None,None,Anytown,WA,00000,0_12634
4,simulated_w2_1099_9509,simulated_taxes_48628,324-30-7438,Anthony,K,Lewi,2705,shattalon dr,None,None,Anytown,WA,00000,0_12634
5,simulated_w2_1099_9510,simulated_taxes_48629,324-30-7438,Christina,S,Do,2705,shattalon dr,None,None,Anytown,WA,00000,0_19345
6,simulated_w2_1099_9511,simulated_taxes_48630,324-30-7438,Christina,S,Do,2705,shattalon dr,None,None,Anytown,WA,00000,0_19345
7,simulated_w2_1099_9512,simulated_taxes_48631,324-30-7438,Christina,S,Do,2705,shattalon dr,None,None,Anytown,WA,00000,0_19345
8,simulated_w2_1099_9513,simulated_taxes_48632,324-30-7438,Willie,H,Do,2705,shattalon dr,None,None,Anytown,WA,00000,0_19348
9,simulated_w2_1099_9514,simulated_taxes_48633,324-30-7438,Willie,H,Do,2705,shattalon dr,None,None,Anytown,WA,00000,0_19348


Simulated SSA records with the most collisions:


,source_record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,simulant_id
0,simulated_ssa_numident_2961,324-30-7438,Anthony,Keith,Lewis,19580828,Male,creation,19580828,0_12634


In [65]:
simulated_geobase_reference_file_ground_truth

,record_id,simulant_id,n_unique_simulants
0,simulated_geobase_reference_file_24949,0_730,1
1,simulated_geobase_reference_file_25721,0_1366,1
2,simulated_geobase_reference_file_25722,0_1366,1
3,simulated_geobase_reference_file_28256,0_1812,1
4,simulated_geobase_reference_file_25723,0_3267,1
...,...,...,...
32683,simulated_geobase_reference_file_19127,0_14659,1
32684,simulated_geobase_reference_file_20551,0_703,1
32685,simulated_geobase_reference_file_20879,0_16072,1
32686,simulated_geobase_reference_file_29861,0_23240,2


In [66]:
%xdel simulated_taxes
%xdel simulated_taxes_source_record_pairs

### Get ground truth by SSN

In [67]:
simulated_ssn_simulant_pairs = persist(
    concat_avoid_fragmentation([
        simulated_census_numident[["record_id", "ssn"]].merge(simulated_census_numident_ground_truth, on="record_id"),
        simulated_alternate_name_numident[["record_id", "ssn"]].merge(simulated_alternate_name_numident_ground_truth, on="record_id"),
        simulated_alternate_dob_numident[["record_id", "ssn"]].merge(simulated_alternate_dob_numident_ground_truth, on="record_id"),
        simulated_name_dob_reference_file[["record_id", "ssn"]].merge(simulated_name_dob_reference_file_ground_truth, on="record_id"),
        simulated_geobase_reference_file[["record_id", "ssn"]].merge(simulated_geobase_reference_file_ground_truth, on="record_id"),
    ])
        [['ssn', 'simulant_id']]
        .pipe(drop_duplicates)
)
simulated_ssn_simulant_pairs

,ssn,simulant_id
11,027-46-6956,0_12990
19,182-87-9730,0_5212
26,614-82-7024,0_12734
29,331-74-9303,0_7558
31,044-86-3127,0_3858
...,...,...
32681,977-84-6791,0_22674
32682,997-95-0405,0_21507
32685,939-53-1702,0_7328
32686,993-56-2286,0_22299


In [68]:
# How many simulated SSNs have different simulant IDs
# contributing to them?
compute((groupby_agg_small_groups(simulated_ssn_simulant_pairs, by='ssn', agg_func=lambda x: x.simulant_id.nunique()) > 1).sum())

1113

## Save results

In [69]:
files = {
    'simulated_census_numident': (simulated_census_numident, simulated_census_numident_ground_truth),
    'simulated_alternate_name_numident': (simulated_alternate_name_numident, simulated_alternate_name_numident_ground_truth),
    'simulated_alternate_dob_numident': (simulated_alternate_dob_numident, simulated_alternate_dob_numident_ground_truth),
    'simulated_geobase_reference_file': (simulated_geobase_reference_file, simulated_geobase_reference_file_ground_truth),
    'simulated_name_dob_reference_file': (simulated_name_dob_reference_file, simulated_name_dob_reference_file_ground_truth),
}

In [70]:
simulated_reference_files = [
    simulated_census_numident,
    simulated_alternate_name_numident,
    simulated_alternate_dob_numident,
    simulated_geobase_reference_file,
    simulated_name_dob_reference_file,
]
# TODO: Rename the ssn column to explicitly include itins, since this is confusing
simulated_all_ssns_itins_in_reference_files = concat_avoid_fragmentation([df[["ssn"]] for df in simulated_reference_files], ignore_index=True)
simulated_ssn_to_pik = (
    simulated_all_ssns_itins_in_reference_files
        .pipe(drop_duplicates)
        .pipe(add_unique_id_col, col_name='pik')
        [['ssn', 'pik']]
)
simulated_ssn_to_pik

,ssn,pik
0,027-46-6956,87174
1,182-87-9730,87181
2,614-82-7024,87185
3,331-74-9303,87186
4,044-86-3127,87188
...,...,...
18974,977-84-6791,106679
18975,997-95-0405,106680
18976,939-53-1702,106681
18977,993-56-2286,106682


In [71]:
simulated_pik_simulant_pairs = (
    simulated_ssn_simulant_pairs
        .merge(simulated_ssn_to_pik, on="ssn", how="inner")
        [['pik', 'simulant_id']]
)
simulated_pik_simulant_pairs

,pik,simulant_id
0,87174,0_12990
1,87181,0_5212
2,87185,0_12734
3,87186,0_7558
4,87188,0_3858
...,...,...
20139,106679,0_22674
20140,106680,0_21507
20141,106681,0_7328
20142,106682,0_22299


In [72]:
%xdel simulated_ssn_simulant_pairs

In [73]:
for file_name, (file, ground_truth) in files.items():
    file = persist(file.merge(simulated_ssn_to_pik, on='ssn', how='left'))
    assert compute(file.pik.notnull().all())

    save_file_with_ground_truth(file_name, file, ground_truth)

In [74]:
simulated_pik_simulant_pairs_path = f'{output_dir}/{data_to_use}/simulated_pik_simulant_pairs.parquet'
remove_path(simulated_pik_simulant_pairs_path)
simulated_pik_simulant_pairs.to_parquet(simulated_pik_simulant_pairs_path)

In [75]:
! date

Mon Nov 20 10:15:07 AM PST 2023
